In [46]:
##Importing ML Libraries
import pandas as pd
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report

In [47]:
##Importing Keras libraries
from keras.models import Model
from keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SimpleRNN
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras import layers

In [48]:
##Read data
data = pd.read_csv(r"C:\Users\19372\Downloads\train.csv")

In [49]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [50]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [51]:
##Preprocessing steps
punct_remove = string.punctuation+string.digits
STOPWORDS = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [52]:
##Applying pre-processing steps to the Text column
for i in range(len(data)):
    data["text"] = data["text"].str.lower()
    data["text"][i] = " ".join([word for word in str(data["text"][i]).split() if word not in STOPWORDS])
    data["text"][i] = data["text"][i].translate(str.maketrans(' ', ' ', punct_remove))
    data["text"][i] = " ".join([lemmatizer.lemmatize(word) for word in data["text"][i].split()])
    data["text"][i] = " ".join([word for (word, pos) in nltk.pos_tag(nltk.word_tokenize(data["text"][i])) if pos[:2] == 'NN' or pos[:2] == 'JJ'])

C:\Users\19372\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\19372\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\19372\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\19372\Anaconda3\lib\site-packages\ipykernel_laun

In [53]:
data["text"]

0                        deed reason earthquake forgive u
1                        forest fire la ronge sask canada
2       resident shelter place notified officer evacua...
3             people wildfire evacuation order california
4            sent photo ruby alaska wildfire pours school
                              ...                        
7608    giant crane bridge collapse nearby home httptc...
7609    ariaahrary thetawniest control wild fire calif...
7610              m utckm volcano hawaii httptcozdtoydebj
7611    police ebike car little portugal ebike rider s...
7612    latest home northern california wildfire abc n...
Name: text, Length: 7613, dtype: object

In [54]:
##Splitting data into train and test
X = data['text']
Y = data['target']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

## Logistic Regression (ML Method)


In [55]:
##Tf-idf vectorization and fitting logistic regression model
tfidf_transformer = TfidfVectorizer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train)
LR = LogisticRegression().fit(X_train_tfidf, y_train)

In [56]:
y_pred_train = LR.predict(X_train_tfidf)

In [57]:
confusion_matrix(y_train, y_pred_train)

array([[3381,  108],
       [ 616, 1985]], dtype=int64)

In [58]:
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.85      0.97      0.90      3489
           1       0.95      0.76      0.85      2601

    accuracy                           0.88      6090
   macro avg       0.90      0.87      0.87      6090
weighted avg       0.89      0.88      0.88      6090



In [63]:
##Predicting the test data
features_test = tfidf_transformer.transform(X_test)
y_pred = LR.predict(features_test)

In [64]:
##confusion matrix for actual vs predicted values
confusion_matrix(y_test, y_pred)

array([[763,  90],
       [216, 454]], dtype=int64)

In [65]:
##Classification Report for actual vs predicted
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.89      0.83       853
           1       0.83      0.68      0.75       670

    accuracy                           0.80      1523
   macro avg       0.81      0.79      0.79      1523
weighted avg       0.80      0.80      0.80      1523



In [66]:
##Accuracy score for ML model (Logistic Regression)
accuracy_score(y_test, y_pred)

0.799080761654629

## Deep Learning

In [67]:
max_features = 10000  
max_len = 1000 
batch_size = 32
max_words = 5000

In [68]:
##padding train dataset
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [69]:
##padding test data
sequences_test = tok.texts_to_sequences(X_test)
sequences_matrix_test = sequence.pad_sequences(sequences_test,maxlen=max_len)

In [70]:
##LSTM model 
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(sequences_matrix, y_train,
                    epochs=5,
                    batch_size=128,
                    validation_split=0.2)

Epoch 1/5
39/39 [==============================] - 34s 882ms/step - loss: 0.6548 - acc: 0.6026 - val_loss: 0.7173 - val_acc: 0.5772
Epoch 2/5
39/39 [==============================] - 28s 711ms/step - loss: 0.5620 - acc: 0.7391 - val_loss: 0.9656 - val_acc: 0.6026
Epoch 3/5
39/39 [==============================] - 44s 1s/step - loss: 0.4604 - acc: 0.8062 - val_loss: 0.5117 - val_acc: 0.7594
Epoch 4/5
39/39 [==============================] - 45s 1s/step - loss: 0.3736 - acc: 0.8452 - val_loss: 0.5945 - val_acc: 0.7504
Epoch 5/5
39/39 [==============================] - 46s 1s/step - loss: 0.3291 - acc: 0.8594 - val_loss: 0.5012 - val_acc: 0.7734


In [71]:
##Evaluating model on the test data
accr = model.evaluate(sequences_matrix_test,y_test)

48/48 [==============================] - 4s 92ms/step - loss: 0.4444 - acc: 0.8011


In [72]:
y_pred=model.predict_classes(sequences_matrix_test)

In [73]:
confusion_matrix(y_test,y_pred)

array([[751, 102],
       [201, 469]], dtype=int64)

In [74]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.79      0.88      0.83       853
           1       0.82      0.70      0.76       670

    accuracy                           0.80      1523
   macro avg       0.81      0.79      0.79      1523
weighted avg       0.80      0.80      0.80      1523



In [75]:
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(sequences_matrix, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

Epoch 1/10
16/39 [===========>..................] - ETA: 7s - loss: 0.6694 - acc: 0.5903

KeyboardInterrupt: 

In [42]:
y_pred=model.predict_classes(sequences_matrix_test)

In [44]:
print(confusion_matrix(y_pred, y_test))

[[738 209]
 [115 461]]


In [45]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      0.87      0.82       853
           1       0.80      0.69      0.74       670

    accuracy                           0.79      1523
   macro avg       0.79      0.78      0.78      1523
weighted avg       0.79      0.79      0.78      1523

